##### 问题:
我们想编写一个可接受参数的装饰器函数。

##### 解决方案:
让我们用一个例子来说明接受参数的过程。假设我们想编写一个为函数添加日志功能
的装饰器，但是又允许用户指定日志的等级以及一些其他的细节作为参数。下面是定
义这个装饰器的可能做法：

In [7]:
from functools import wraps
import logging
def logged(level, name=None, message=None):
    def decorate(func):
        logname = name if name else func.__module__
        log = logging.getLogger(logname)
        logmsg = message if message else func.__name__
        @wraps(func)
        def wrapper(*args, **kwargs):
            log.log(level, logmsg)   #不知道log有什么用
            return func(*args, **kwargs)
        return wrapper
    return decorate

# Example use
@logged(logging.DEBUG)
def add(x, y):
    return x + y

@logged(logging.CRITICAL, 'example')
def spam():
    print('Spam!')


print(add(2,3))
spam()  #输出两个spam一个是log.log输出的,一个是spam函数的print

5
Spam!


初看上去这个实现显得很有技巧性，但其中的思想相对来说是很简单的。最外层的
logged()函数接受所需的参数，并让它们对装饰器的内层函数可见。内层的 decorate()
函数接受一个函数并给它加上一个包装层。关键部分在于这个包装层可以使用传递给
logged()的参数。

编写一个可接受参数的装饰器是需要一些技巧的，因为这会涉及底层的调用顺序。具
体来说，如果有这样的代码：

In [8]:
'''
@decorator(x, y, z)
def func(a, b):
    pass
'''

'\n@decorator(x, y, z)\ndef func(a, b):\n    pass\n'

装饰的过程会按照下列方式来进行：

In [9]:
'''
def func(a, b):
    pass
func = decorator(x, y, z)(func) 

'''

'\ndef func(a, b):\n    pass\nfunc = decorator(x, y, z)(func) \n\n'

请仔细观察，decorator(x, y, z)的结果必须是一个可调用对象，这个对象反过来接受一
个函数作为输入，并对其进行包装。请参见 9.7 节中另一个关于让装饰器接受参数的
例子。